In [1]:
print("Hii")

Hii


In [4]:
from dotenv import load_dotenv
import os
from neo4j import GraphDatabase
from langchain.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain

# Load environment variables from .env file
load_dotenv()

# Get Neo4j credentials from environment variables
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

def test_direct_connection():
    """Test direct Neo4j connection using the Python driver"""
    try:
        # Create a driver instance
        driver = GraphDatabase.driver(
            NEO4J_URI,
            auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
        )
        
        # Verify connectivity
        driver.verify_connectivity()
        print("Direct Neo4j connection successful!")
        
        return driver
    
    except Exception as e:
        print(f"Error connecting to Neo4j: {e}")
        return None
    
def test_langchain_connection():
    """Test Neo4j connection using LangChain"""
    try:
        # Initialize Neo4jGraph with credentials
        graph = Neo4jGraph(
            url=NEO4J_URI,
            username=NEO4J_USERNAME,
            password=NEO4J_PASSWORD
        )
        
        # Test a simple query
        result = graph.query("MATCH (n) RETURN count(n) as count")
        print("LangChain Neo4j connection successful!")
        print(f"Number of nodes in database: {result[0]['count']}")
        
        return graph
    
    except Exception as e:
        print(f"Error connecting via LangChain: {e}")
        return None

# if __name__ == "__main__":
# Test both connection methods
direct_driver = test_direct_connection()
langchain_graph = test_langchain_connection()

# Clean up direct driver connection if it was successful
if direct_driver:
    direct_driver.close()

Direct Neo4j connection successful!


/tmp/ipykernel_19428/3585152814.py:38: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


LangChain Neo4j connection successful!
Number of nodes in database: 0


In [ ]:
import requests
import json
import time
import sseclient

# MCP server URL
MCP_URL = "http://localhost:8080"  # Change to your MCP server address

def upload_markdown_file_mcp(file_path, group_id=None):
    """
    Upload a Markdown file to Graphiti MCP server
    """
    # Read the Markdown file
    with open(file_path, 'r') as file:
        content = file.read()
    
    # Create MCP message
    mcp_message = {
        "type": "episode",
        "operation": "create",
        "data": {
            "content": content,
            "content_type": "text/markdown",
            "source": os.path.basename(file_path)
        }
    }
    
    # Add group if specified
    if group_id:
        mcp_message["data"]["group_id"] = group_id
    
    # Send to MCP server
    response = requests.post(
        f"{MCP_URL}/v1/messages",
        json=mcp_message,
        headers={"Content-Type": "application/json"}
    )
    
    if response.status_code != 200:
        print(f"Error uploading file: {response.text}")
        return None
    
    result = response.json()
    episode_id = result.get("data", {}).get("id")
    print(f"Episode created with ID: {episode_id}")
    return episode_id

In [ ]:
# # System Prompt for Graphiti-Enabled Agent

# You are an AI assistant with access to Graphiti's knowledge graph capabilities. You can maintain context and knowledge across conversations using the following tools:

# ## Your Available Tools:

# 1. `add_episode`: Store information in the knowledge graph
#    - Use this to save conversations, requirements, or any important information
#    - Example: Recording a customer meeting, technical requirements, or project details
#    ```python
#    add_episode(
#        name="Meeting with Client X",
#        episode_body="Client discussed requirements...",
#        source="text"
#    )
#    ```

# 2. `search_nodes`: Find relevant information
#    - Use this to find previously stored information
#    - Example: Looking up past conversations or requirements
#    ```python
#    search_nodes(
#        query="security requirements for Project X"
#    )
#    ```

# 3. `search_facts`: Find relationships between entities
#    - Use this to understand connections between different pieces of information
#    - Example: Finding connections between requirements and projects

# 4. `get_episodes`: Retrieve recent conversations/information
#    - Use this to review recent interactions
#    - Example: Getting last 5 conversations about a specific topic

# ## Your Responsibilities:

# 1. Information Storage:
#    - Store important information from conversations
#    - Record requirements, preferences, and procedures
#    - Maintain context across interactions

# 2. Information Retrieval:
#    - Search for relevant past information when needed
#    - Connect related pieces of information
#    - Provide context from previous interactions

# 3. Knowledge Organization:
#    - Use appropriate group_ids to organize related information
#    - Structure information clearly and consistently
#    - Link related pieces of information

# ## When to Use Each Tool:

# USE `add_episode` WHEN:
# - Recording new conversations
# - Storing requirements or specifications
# - Saving important decisions or outcomes
# - Documenting technical details

# USE `search_nodes` WHEN:
# - Looking for specific information
# - Finding relevant past conversations
# - Checking existing requirements
# - Verifying previous decisions

# USE `search_facts` WHEN:
# - Understanding relationships between entities
# - Finding connections between requirements
# - Tracing decision dependencies
# - Mapping project relationships

# USE `get_episodes` WHEN:
# - Reviewing recent conversations
# - Checking latest updates
# - Following up on previous discussions

# ## Information Structure Guidelines:

# When adding episodes, structure information like this:

# FOR CONVERSATIONS:
# ```text
# Topic: [Main Topic]
# Participants: [Who was involved]
# Key Points:
# - [Point 1]
# - [Point 2]
# Decisions Made:
# - [Decision 1]
# - [Decision 2]
# Next Steps:
# - [Action item 1]
# - [Action item 2]
# ```

# FOR REQUIREMENTS:
# ```text
# Requirement Type: [Technical/Business/Security]
# Description: [Clear description]
# Priority: [High/Medium/Low]
# Dependencies: [Any dependencies]
# Stakeholders: [Who is involved]
# ```

# FOR TECHNICAL DOCUMENTATION:
# ```text
# Component: [Name]
# Purpose: [Description]
# Technical Details:
# - [Detail 1]
# - [Detail 2]
# Dependencies: [List]
# ```

# ## Best Practices:

# 1. Always add context when storing information
# 2. Use clear, descriptive names for episodes
# 3. Include relevant metadata (dates, participants, etc.)
# 4. Link related information using consistent group_ids
# 5. Structure information for easy retrieval
# 6. Include key terms for better searchability

# ## Response Format:

# When responding to queries:
# 1. First, search for relevant context using appropriate tools
# 2. Combine information from multiple sources if needed
# 3. Present information clearly and structured
# 4. Store new important information from the interaction
# 5. Link new information to existing knowledge

# Remember: Your goal is to maintain a coherent knowledge graph that helps provide better context and more informed responses across conversations.

In [1]:
from langgraph.prebuilt import create_react_agent


In [4]:
import os
import asyncio
import argparse
from typing import Dict, Any, Optional, List
from dotenv import load_dotenv
import mimetypes
import pathlib

# Load environment variables
load_dotenv()

# LangChain MCP adapter imports
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage

class DocumentProcessor:
    """Agent for processing documents and storing knowledge using Graphiti"""
    
    def __init__(self, model_name: str = "gpt-4"):
        self.model_name = model_name
        self.llm = None
        self.agent = None
        self.mcp_client = None
        self.system_prompt = """
        You are an AI assistant with access to both document processing and knowledge graph capabilities.
        
        Available Tools:
        1. Document Processing (MarkItDown):
           - Process various document formats (PDF, PPT, Word, Excel, etc.)
           - Extract text from images using OCR
           - Transcribe audio files
           - Parse structured formats (CSV, JSON, XML)
           - Process web content and YouTube URLs
        
        2. Knowledge Graph (Graphiti):
           - add_episode: Store processed information
           - search_nodes: Find relevant information
           - search_facts: Find relationships
           - get_episodes: Retrieve conversation history
        
        Your responsibilities:
        1. Process uploaded documents effectively
        2. Extract meaningful information
        3. Store processed content in the knowledge graph
        4. Maintain relationships between documents and information
        5. Provide structured access to stored knowledge
        """
    
    async def setup(self):
        """Set up the agent with both MarkItDown and Graphiti tools."""
        self.llm = ChatOpenAI(model=self.model_name, api_key=os.getenv("OPENAI_API_KEY"))
        
        try:
            # Connect to both MCP servers
            self.mcp_client = MultiServerMCPClient(
                {
                    "graphiti": {
                        "url": "http://localhost:8000/sse",
                        "transport": "sse",
                    },
                    "markitdown": {
                        "url": "http://127.0.0.1:3001/sse",
                        "transport": "sse",
                    }
                }
            )
            
            await self.mcp_client.__aenter__()
            mcp_tools = self.mcp_client.get_tools()
            print(f"Loaded {len(mcp_tools)} tools from MCP servers")
            
            self.agent = create_react_agent(
                self.llm,
                mcp_tools,
                prompt=self.system_prompt
            )
            
        except Exception as e:
            print(f"Error connecting to MCP servers: {str(e)}")
            raise

    async def process_document(self, file_path: str) -> Dict[str, Any]:
        """Process a document and store its information.
        
        Args:
            file_path: Path to the document to process
            
        Returns:
            Processing and storage results
        """
        if not self.agent:
            await self.setup()
        
        try:
            # Get file type
            mime_type, _ = mimetypes.guess_type(file_path)
            file_ext = pathlib.Path(file_path).suffix.lower()
            
            process_prompt = f"""
            Please process this document and store its information:
            
            File: {file_path}
            Type: {mime_type or file_ext}
            
            1. Use appropriate MarkItDown tools to process the document
            2. Extract key information and insights
            3. Store the processed information in the knowledge graph using add_episode
            4. Create appropriate relationships using search_facts if relevant
            
            Please provide a summary of what was processed and stored.
            """
            
            result = await self.agent.ainvoke({
                "messages": [
                    HumanMessage(content=process_prompt)
                ]
            })
            
            return result
            
        except Exception as e:
            return {
                "error": f"Error processing document: {str(e)}"
            }

    async def search_documents(self, query: str, doc_types: Optional[List[str]] = None) -> Dict[str, Any]:
        """Search for information across processed documents.
        
        Args:
            query: Search query
            doc_types: Optional list of document types to search within
            
        Returns:
            Search results
        """
        if not self.agent:
            await self.setup()
        
        try:
            type_filter = f" within {', '.join(doc_types)}" if doc_types else ""
            search_prompt = f"""
            Please search for information{type_filter} about:
            
            {query}
            
            1. Use search_nodes to find relevant documents and information
            2. Use search_facts to find relationships between documents
            3. Compile the results in a clear, structured format
            """
            
            result = await self.agent.ainvoke({
                "messages": [
                    HumanMessage(content=search_prompt)
                ]
            })
            
            return result
            
        except Exception as e:
            return {
                "error": f"Error searching documents: {str(e)}"
            }

    async def close(self):
        """Clean up resources."""
        if self.mcp_client:
            await self.mcp_client.__aexit__(None, None, None)


async def main():
    """Run the document processor."""
    parser = argparse.ArgumentParser(description="Document Processor with Knowledge Graph")
    parser.add_argument("--action", choices=["process", "search"], required=True,
                      help="Action to perform (process or search)")
    parser.add_argument("--file", help="Path to file to process")
    parser.add_argument("--query", help="Search query")
    parser.add_argument("--doc-types", nargs="+", help="Document types to search within")
    parser.add_argument("--model", default="gpt-4", help="LLM model to use")
    args = parser.parse_args()
    
    if not os.getenv("OPENAI_API_KEY"):
        print("ERROR: OPENAI_API_KEY not found in environment variables")
        print("Please set it in your .env file")
        return
    
    processor = DocumentProcessor(model_name=args.model)
    
    try:
        if args.action == "process":
            if not args.file:
                print("ERROR: --file is required for process action")
                return
            result = await processor.process_document(args.file)
        else:  # search
            if not args.query:
                print("ERROR: --query is required for search action")
                return
            result = await processor.search_documents(args.query, args.doc_types)
        
        if "error" in result:
            print(f"Error: {result['error']}")
            return
        
        # Display the result
        ai_messages = [msg for msg in result["messages"] if isinstance(msg, AIMessage)]
        if ai_messages:
            print("\n===== PROCESSING RESULT =====")
            print(ai_messages[-1].content)
        else:
            print("No result was generated")
    
    finally:
        await processor.close()


if __name__ == "__main__":
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop